# Creating a new FEBio input file: FEB

## The basics

We can create a new FEB file with basically no information. To do so, we can simply call Feb and provide it a version. Right now, we accept versions 2.5 and 3.0.

In [ ]:
from febio_python import Feb

feb = Feb(version=2.5)
feb

You will notices that all contents of the feb file are zero. This means that there is nothing stored in the feb right now. Let's start modifying it. We will begin with simple structures, like module, globals and controls:

In [ ]:
feb.setup_module(module_type="solid") # default values
feb.setup_globals(T=0, R=0, Fc=0) # default values
feb.setup_controls(analysis_type="static") # here, you can change basic settings. See docs for more info.
feb.setup_output(variables=["displacement", "Lagrange strain", "stress"]) # default values

We can check the FEB object. It now has some data:

In [ ]:
feb

## Adding Mesh

We will now add more interestind data to our feb. Let's start by creating a simple mesh with pyvista:

In [ ]:
import pyvista as pv
pv.set_jupyter_backend('client')

# Create a simple mesh (plane mesh)
mesh = pv.Plane(direction=(0,0,1), i_size=2, j_size=1, i_resolution=6, j_resolution=3)
mesh = mesh.cast_to_unstructured_grid() # we will be using unstructured grid for FEBio
mesh.plot(show_edges=True, jupyter_backend='client', window_size=(800, 400), cpos='xy')

In Pyvista, an unstructured grid is defined as "points" (nodes) and "cells" (elements). We can access them like this: 

In [ ]:
mesh.points

In [ ]:
mesh.cells_dict

In [ ]:
print(f"Number of nodes: {mesh.points.shape[0]}")
print(f"Number of elements: {mesh.cells_dict[pv.CellType.QUAD].shape[0]}")

In [ ]:
from febio_python.core import Nodes, Elements
# Create nodes
nodes = Nodes(name="plane", coordinates=mesh.points)
# Create elements
elements = Elements(name="plane_elements",
                    mat="1",
                    part=None,
                    type="QUAD",
                    connectivity=mesh.cells_dict[pv.CellType.QUAD.value],
                    )
# Add nodes and elements to the feb object (need to be lists)
feb.add_nodes([nodes])
feb.add_elements([elements])

We can now see that we have two 'Geometry' items:

In [ ]:
feb

We can further inspect them:

In [ ]:
feb.inspect_nodes_and_elements()

## Modfying nodes/elements

Oh no! We have defined a too coarse mesh! Let's create a fine mesh and update our feb.

In [ ]:
# Create a simple mesh (plane mesh)
mesh = pv.Plane(direction=(0,0,1), i_size=2, j_size=1, i_resolution=10, j_resolution=5)
mesh = mesh.cast_to_unstructured_grid() # we will be using unstructured grid for FEBio
mesh.plot(show_edges=True, jupyter_backend='client', window_size=(800, 400), cpos='xy')

In [ ]:
print(f"Number of nodes: {mesh.points.shape[0]}")
print(f"Number of elements: {mesh.cells_dict[pv.CellType.QUAD].shape[0]}")

Note that if we use the function "ADD" again, it will add the new nodes to the existing nodes in the same mesh. This is used when we are creating mesh in an iterative process. Let's try:

In [ ]:
# Create nodes
nodes = Nodes(name="plane", coordinates=mesh.points)
# Create elements
elements = Elements(name="plane_elements",
                    mat="1",
                    part=None,
                    type="QUAD",
                    connectivity=mesh.cells_dict[pv.CellType.QUAD.value],
                    )
# Add nodes and elements to the feb object (need to be lists)
feb.add_nodes([nodes])
feb.add_elements([elements])
# Inspect nodes and elements
feb.inspect_nodes_and_elements()

You can see that it had inscrease the number of nodes/elements. But this is not what we are looking for.
Let's clean the feb data and add it again:

In [ ]:
feb.clear_nodes()
feb.clear_elements()
feb

In [ ]:
feb.add_nodes([nodes])
feb.add_elements([elements])
feb.inspect_nodes_and_elements()

Is there another method? We need triangle elements!! Also, What if we have multiple nodes or elements and we do not want to delete all of them and just update the existing ones? Let's create another mesh and update the 'plane' mesh in the feb file.

In [ ]:
# Create a simple mesh (plane mesh)
mesh = pv.Plane(direction=(0,0,1), i_size=2, j_size=1, i_resolution=20, j_resolution=10)
mesh = mesh.triangulate() # we will be using unstructured grid for FEBio
mesh = mesh.cast_to_unstructured_grid() # we will be using unstructured grid for FEBio
mesh.plot(show_edges=True, jupyter_backend='client', window_size=(800, 400), cpos='xy')

In [ ]:
print(f"Number of nodes: {mesh.points.shape[0]}")
print(f"Number of elements: {mesh.cells_dict[pv.CellType.TRIANGLE].shape[0]}")

In [ ]:
# Create nodes
nodes = Nodes(name="plane", coordinates=mesh.points)
# Create elements
elements = Elements(name="plane_elements",
                    mat="1",
                    part=None,
                    type="TRIANGLE",
                    connectivity=mesh.cells_dict[pv.CellType.TRIANGLE.value],
                    )
# Add nodes and elements to the feb object (need to be lists)
feb.update_nodes([nodes])
feb.update_elements([elements])
# Inspect nodes and elements
feb.inspect_nodes_and_elements()

## Adding Load

Let's try adding a shear load to the mesh:

In [ ]:
import numpy as np
from febio_python.core import NodalLoad, LoadCurve, NodeSet

# First, let's select the nodeset to apply the load.
# we will add load to the nodes at the right edge of the mesh
x_values = mesh.points[:, 0]
selected_nodes = np.where(x_values == x_values.max())[0]

# Create a nodeset
nodeset = NodeSet(name="right_edge", ids=selected_nodes)
# Create a nodal load
shear_load = NodalLoad(node_set="right_edge",
                       dof="y",
                       scale=-25.0, # negative value means force is pointing in the negative direction
                       load_curve=1,
                       )
# Create a load curve
load_curve = LoadCurve(id=1, interpolate_type="smooth", data=[(0, 0), (1, 1)])

# Add nodeset, nodal load and load curve to the feb object
feb.add_nodesets([nodeset])
feb.add_nodal_loads([shear_load])
feb.add_loadcurves([load_curve])
feb

Let's try to add a non-linear load now. Pointing in the x-direction

In [ ]:
# first, get the y position of the nodes
y_values = mesh.points[selected_nodes, 1]
# then, normalize the y values, so that we can use them as 'maps'
y_values = (y_values - y_values.min()) / (y_values.max() - y_values.min())
# now, create a normal distribution curve using the y values, centered at 
# middle of the y values (0.5)
tensile_load_map = np.exp(-((y_values - 0.5) ** 2) / 0.1)
# plot the load curve
import matplotlib.pyplot as plt
plt.plot(y_values, tensile_load_map)
plt.xlabel("Normalized y position")
plt.ylabel("Load curve")
plt.title("Load curve for the shear load")
plt.show()

# Create a nodal load
tensile_load = NodalLoad(node_set="right_edge",
                       dof="x",
                       scale=100.0*tensile_load_map, # 
                       load_curve=1,
                       )
feb.add_nodal_loads([tensile_load])
feb

## Add boundary condition

Now, let's fix the left boundary of the mesh. We will be applying fixed condition to restrain the mesh in all coordinates. In addition, since this is a simple plane mesh, we will apply constraint in z (for all nodes) and rotation (shell constrain) for all the left nodes.

In [ ]:
from febio_python.core import FixCondition

# Fix the left edge of the mesh
x_values = mesh.points[:, 0]
selected_nodes = np.where(x_values == x_values.min())[0]
all_nodes = np.arange(mesh.points.shape[0]) # used to fix all nodes in the z direction
# Create nodesets
left_nodeset = NodeSet(name="left_edge", ids=selected_nodes)
all_nodeset = NodeSet(name="all_nodes", ids=all_nodes)
# Create a fix condition
left_fix_condition = FixCondition(dof="x,y,z,sx,sy", node_set="left_edge")
# we will 
all_fix_condition = FixCondition(dof="z", node_set="all_nodes")

# Add nodeset and fix condition to the feb object
feb.add_nodesets([left_nodeset, all_nodeset])
feb.add_boundary_conditions([left_fix_condition, all_fix_condition])

In [ ]:
feb

## Adding Material

Now, let's add a material. We will use simple Isotropic-Elastic.

In [ ]:
from febio_python.core import Material

mat = Material(
    id=1,
    type="isotropic elastic",
    name="plane material",
    parameters=dict(
        E=1e6,
        v=0.3,
        density=1,
    )
)

feb.add_materials([mat])

# Add Element data

In [ ]:
from febio_python.core import ElementData

shell_thickness = ElementData(
    name="Element thickness",
    var="shell thickness",
    elem_set="plane_elements",
    data=np.full((mesh.n_cells, 3), 0.01),
    ids=np.arange(0, mesh.n_cells + 1),
)
feb.add_element_data([shell_thickness])

# Running FEB

In [ ]:
from febio_python.feb import run
# Save the FEB file
feb.write("plane_mesh.feb")
run("plane_mesh.feb")

# Reading XPLT

In [ ]:
from febio_python import Xplt

xplt = Xplt("plane_mesh.xplt")
xplt

## Example properties

In [ ]:
xplt.nodes # list of Nodes objects

In [ ]:
xplt.elements # list of Elements objects

In [ ]:
xplt.states # list of States objects

## States data:

In [ ]:
xplt.states.nodes[0]

In [ ]:
xplt.states.elements[0]

# FEBio Container

In [ ]:
from febio_python import FEBioContainer

container = FEBioContainer(feb=feb,
                           xplt=xplt
                           )

Can also load from a file directly:

In [ ]:
container = FEBioContainer(feb="plane_mesh.feb",
                           xplt="plane_mesh.xplt"
                           )

In [ ]:
container.feb

In [ ]:
container.xplt

In [ ]:
container.nodes

# Ploting Feb or Xplt

In [ ]:
from febio_python.utils.pyvista_utils import febio_to_pyvista

grids_list = febio_to_pyvista(container)
len(grids_list)

In [ ]:
grids_list[-1].plot(cpos='xy', show_edges=True, scalars="stress")

You can also quickly convert all cell data to node data:

In [ ]:
last_grid_as_nodal_data = grids_list[-1].cell_data_to_point_data()
last_grid_as_nodal_data.plot(cpos='xy', show_edges=True, scalars="stress")

In [ ]:
last_grid_as_nodal_data

When using the FEBio container, we have access to both Feb and Xplt data. This means that we can retrieve nodal load, boundary conditions, etc. Here is a cool plot that we can do:

In [ ]:
plotter = pv.Plotter()
strain_xx = last_grid_as_nodal_data["Lagrange strain"][:, 0]
fixed_nodes = last_grid_as_nodal_data["fix"].sum(1)

plotter.add_mesh(last_grid_as_nodal_data, 
                 scalars=strain_xx, 
                 cmap="coolwarm", 
                 show_edges=True, 
                 scalar_bar_args={"title": "Strain - XX"})
plotter.add_mesh(last_grid_as_nodal_data.points, 
                 scalars=fixed_nodes, 
                 cmap="viridis",
                 style="points", 
                 point_size=10, 
                 render_points_as_spheres=True, show_scalar_bar=False)
plotter.add_arrows(last_grid_as_nodal_data.points, 
                   last_grid_as_nodal_data["nodal_load"], 
                   mag=5e-3, # This controls the mag of the arrows. Not the actual load. There may be a better way to control this.
                   show_scalar_bar=False,
                   color="orange")
plotter.show(cpos="xy")
